In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mediumreview/dataset.csv


In [2]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 48.7 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=f82347e1a70eb7f0c520f17d34282385810a8469844060085802bada1fe66e09
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=7db27aca285cf642027cad0d392137beaece3c2bdf3e3fa9ff2c7b28f0e11523
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252b

I'm going to import the necessary library.

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from simpletransformers.classification import ClassificationModel
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

Let's import the dataset

In [4]:
df = pd.read_csv('/kaggle/input/mediumreview/dataset.csv')

Let's check the first ten rows of the dataset.

In [5]:
df.head(10)

,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,predicted_category,sentiment
0,326d4bdd-8fc6-4d64-b3bf-ce393fb2ae9b,woww,5,0,NaN,2022-08-16 05:55:38,NaN,NaN,USER_EXPERIENCE,NEUTRAL
1,4bbf741b-2f69-43cd-bb76-f9b5f84c83b5,let me know more details about this,3,0,4.5.1143533,2022-08-16 04:44:45,NaN,NaN,CONTENT,NEUTRAL
2,3cb1136d-e7c6-4999-aa84-fdc7bcdccf56,i've been using this for a while and there's a...,2,0,4.5.1143533,2022-08-16 04:05:27,NaN,NaN,INTERFACE,NEGATIVE
3,d6f80f05-a6e8-44f3-a380-7c59ed3d208b,good,2,0,4.5.1143533,2022-08-16 00:22:33,NaN,NaN,USER_EXPERIENCE,POSITIVE
4,eecb277c-c658-4b2d-86c9-77d3a7022cac,mjkobe,5,0,4.5.1143533,2022-08-15 20:08:25,NaN,NaN,USER_EXPERIENCE,NEUTRAL
5,738e80f5-ecb7-4489-bd01-fdf9f84cc7d4,good,2,0,4.5.1143533,2022-08-15 20:05:47,NaN,NaN,USER_EXPERIENCE,POSITIVE
6,f190d2a9-b627-4839-ab4a-4b0da9ad260c,awesome for me who likes to read about differe...,5,0,4.5.1143533,2022-08-15 19:29:54,NaN,NaN,CONTENT,POSITIVE
7,015ff161-7372-4bea-a0df-2e7402d1d287,best thing i discovered in my life. that's is ...,5,0,4.5.1143533,2022-08-15 17:44:17,NaN,NaN,USER_EXPERIENCE,POSITIVE
8,48c50ab1-3097-4e27-bb49-15ad818e2c66,good project,5,0,4.5.1143533,2022-08-15 16:45:30,NaN,NaN,USER_EXPERIENCE,POSITIVE
9,f2b514ba-27c4-49da-8831-ddbd9c67287f,"basic search functionality is non existent, no...",1,0,4.5.1143533,2022-08-15 15:46:29,NaN,NaN,INTERFACE,NEGATIVE


Let's check the shape of the dataset

In [6]:
df.shape

(45414, 10)

Based on the output above, the dataset has 45.414 rows and 10 columns.

Let's check the type of each columns.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45414 entries, 0 to 45413
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              45414 non-null  object
 1   content               45414 non-null  object
 2   score                 45414 non-null  int64 
 3   thumbsUpCount         45414 non-null  int64 
 4   reviewCreatedVersion  40367 non-null  object
 5   at                    45414 non-null  object
 6   replyContent          2477 non-null   object
 7   repliedAt             2477 non-null   object
 8   predicted_category    45414 non-null  object
 9   sentiment             45414 non-null  object
dtypes: int64(2), object(8)
memory usage: 3.5+ MB


Let's check if the dataset has null values or not.

In [8]:
df.isna().sum()

reviewId                    0
content                     0
score                       0
thumbsUpCount               0
reviewCreatedVersion     5047
at                          0
replyContent            42937
repliedAt               42937
predicted_category          0
sentiment                   0
dtype: int64

Let's check the count of each values on the columns.

In [9]:
df['content'].value_counts()

good                                                                                                                                                                               1302
nice                                                                                                                                                                                710
great                                                                                                                                                                               493
awesome                                                                                                                                                                             438
great app                                                                                                                                                                           363
                                                                                

In [10]:
df['score'].value_counts()

5    32330
4     6294
1     3647
3     1830
2     1313
Name: score, dtype: int64

In [11]:
df['reviewCreatedVersion'].value_counts()

3.14.1001727    1084
3.14.1001624     842
3.6.1000095      807
3.16.1016100     703
3.10.1001075     701
                ... 
3.6.6002           1
3.6.6210           1
3.6.1000227        1
3.14.1001361       1
3.6.6942           1
Name: reviewCreatedVersion, Length: 368, dtype: int64

In [12]:
df['replyContent'].value_counts()

Thank you for your feedback, we'd love to hear more of this in https://reddit.com/r/MediumApp/.                                                                                                                                    94
Hi there, we're sorry for the frustration around this! The issue with opening the app from external links has been resolved in the latest app update, which has just been released to Google Play. Thank you for your patience!    27
We're sorry -- we let a bug slip out! If you download the latest update it should fix the sign-in problems. Please let us know if this fixes things for you :)                                                                     26
Thanks for the support. We'd love to hear more feedback here (www.reddit.com/r/MediumApp/) on how we can improve our Android app.                                                                                                  24
Thank you for the review!                                                       

In [13]:
df['predicted_category'].value_counts()

USER_EXPERIENCE    20398
CONTENT            14454
INTERFACE           7645
SUBSCRIPTION        2917
Name: predicted_category, dtype: int64

In [14]:
df['sentiment'].value_counts()

POSITIVE    34281
NEUTRAL      5889
NEGATIVE     5244
Name: sentiment, dtype: int64

From the output above, I'm going to use `content` and `sentiment` columns for the training because the model only need to predict the content and the result will be the sentiment.

Since the sentiment values are categorical, I'm going to convert the categorical values into integer label. The `positive` sentiment will be `2`. The `neutral` will be `1`. The `negative` will be `0`.

In [15]:
df['sentiment'] = df['sentiment'].astype('category').cat.codes

In [16]:
df['sentiment'].value_counts()

2    34281
1     5889
0     5244
Name: sentiment, dtype: int64

Let's split the dataset into train and test.

In [17]:
train, test = train_test_split(df, test_size = 0.1, random_state = 101)

In [18]:
train.head()

,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,predicted_category,sentiment
23812,58fa0d6c-6955-4248-bddd-ffb44269421f,excellent,5,0,3.9.1000664,2019-06-20 06:43:22,NaN,NaN,USER_EXPERIENCE,2
44758,6a07c215-ebf5-406e-89a1-c7220f4921ff,super cool,5,0,4.5.1151200,2022-11-19 07:56:28,NaN,NaN,USER_EXPERIENCE,2
28423,0c0615c4-0df0-4add-b994-1abadf03ee04,well m afraid of the 2 times to read an articl...,5,3,3.6.1000181,2019-02-12 14:35:47,NaN,NaN,SUBSCRIPTION,0
25463,9b4a8687-083e-4867-94ca-480fcc46295a,intuitive and clean interface... i like it.,5,2,3.7.1000462,2019-05-02 19:45:20,NaN,NaN,INTERFACE,2
23386,f3ef9af4-532f-4037-972a-dff3be3a9b59,excellent!,5,0,NaN,2019-07-03 13:28:46,NaN,NaN,USER_EXPERIENCE,2


In [19]:
test.head()

,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,predicted_category,sentiment
28116,3febd62e-0b75-4fbf-92e7-aa8a7c67ef24,my favorite app. fantastic collection of writi...,5,2,3.6.1000199,2019-02-20 05:27:57,"Hi Carol, thank you so much for your kind word...",2019-02-20 12:40:54,CONTENT,2
17191,73a1dd9f-e337-42b7-832f-08bf2d7732a0,i am really enjoying medium. there are a lot o...,5,1,3.14.1001727,2020-01-08 19:47:14,NaN,NaN,CONTENT,2
21668,de8222d0-db64-47ac-aa4a-98944506da29,"cool, excellent",4,0,3.10.1001075,2019-08-25 13:10:56,NaN,NaN,USER_EXPERIENCE,2
8987,c4c1e84f-f21b-4ed1-8ee9-02697b7335b5,brilliant app,5,0,3.19.1040254,2020-10-18 10:28:42,NaN,NaN,USER_EXPERIENCE,2
15085,e8799a94-64d6-4d43-862e-03f1ae83365b,great,5,0,3.16.1002166,2020-03-25 08:52:29,NaN,NaN,USER_EXPERIENCE,2


In [20]:
print(len(train))
print(len(test))

40872
4542


Let's drop all the unnecessary columns for train and test. The necessary columns are only `content` and `sentiment`.

In [21]:
train = train[['content', 'sentiment']]

In [22]:
train.head()

,content,sentiment
23812,excellent,2
44758,super cool,2
28423,well m afraid of the 2 times to read an articl...,0
25463,intuitive and clean interface... i like it.,2
23386,excellent!,2


In [23]:
test = test[['content', 'sentiment']]

In [24]:
test.head()

,content,sentiment
28116,my favorite app. fantastic collection of writi...,2
17191,i am really enjoying medium. there are a lot o...,2
21668,"cool, excellent",2
8987,brilliant app,2
15085,great,2


I'm going to tune the hyperparameters by adjusting it in the `model_args`.

In [25]:
model_args = {
    "learning_rate": 1e-5,
    "num_train_epochs": 3,
    "train_batch_size": 32,
    "max_seq_length": 128,
    "reprocess_input_data": True
}

In [26]:
# # Clear output folder in Kaggle
# import os

# def remove_folder_contents(folder):
#     for the_file in os.listdir(folder):
#         file_path = os.path.join(folder, the_file)
#         try:
#             if os.path.isfile(file_path):
#                 os.unlink(file_path)
#             elif os.path.isdir(file_path):
#                 remove_folder_contents(file_path)
#                 os.rmdir(file_path)
#         except Exception as e:
#             print(e)

# folder_path = '/kaggle/working'
# remove_folder_contents(folder_path)
# os.rmdir(folder_path)

Let's build the model using the pre-trained model which is XLNet (based on the sum of last 2 digits of my NIM: 8 + 1 = 9). I'm going to use the early stopping callback to stop the training process if the performance of a traing data has stopped improving.

In [27]:
model = ClassificationModel("xlnet", "xlnet-base-cased", 
                            num_labels = 3, # The class is only 3: positive, neutral, negative
                            args = model_args, # Tune the hyperparameter
                            use_cuda = True) # Use GPU to improve the speed of the training process

early_stopping_cb = EarlyStopping(monitor = 'val_loss', # Stop the training process if the performance of the validation loss has stopped improving
                                  patience = 5, 
                                  verbose = 0, 
                                  mode = 'min',
                                  restore_best_weights = True) # Save the best weights

history = model.train_model(train, callbacks = early_stopping_cb, 
                            overwrite_output_dir = True) # Overwrite the output directory on Kaggle

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

...

  0%|          | 0/40872 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/1278 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/1278 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/1278 [00:00<?, ?it/s]

Let's check the `result`, `model_outputs`, and the `wrong_predictions`.

In [28]:
result, model_outputs, wrong_predictions = model.eval_model(test)

/opt/conda/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:1455: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/4542 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/568 [00:00<?, ?it/s]

In [29]:
result

{'mcc': 0.8449636671109445, 'eval_loss': 0.1992525821158164}

In [30]:
model_outputs

array([[-2.8515625 , -3.15820312,  7.55078125],
       [-2.96875   , -2.91210938,  7.51171875],
       [-2.58789062, -3.35546875,  7.30078125],
       ...,
       [-2.59570312, -3.26171875,  7.17578125],
       [-2.70507812,  0.86474609,  1.82714844],
       [-3.76367188, -1.47363281,  6.44921875]])

To convert the continuous `model_outputs` values into integer labels, I'm going to use the `np.argmax()` function to extract the index of the maximum value in each row of `model_outputs`.

In [31]:
model_outputs = np.argmax(model_outputs, axis = 1) # axis = 1 -> row

In [32]:
model_outputs

array([2, 2, 2, ..., 2, 2, 2])

In [33]:
wrong_predictions

[{'guid': 5, 'text_a': 'good targeting algorithms', 'text_b': None, 'label': 1},
 {'guid': 72, 'text_a': "very good on my moto, but can't load on my lenovo. so, have to use the phone to read member series.", 'text_b': None, 'label': 0},
 {'guid': 76, 'text_a': 'must have', 'text_b': None, 'label': 1},
 {'guid': 93, 'text_a': 'you have to upgrade to premium membership for most of the articles.. not really recommending it, if you are looking for a good free reading app for yourself..', 'text_b': None, 'label': 1},
 {'guid': 101, 'text_a': 'must have', 'text_b': None, 'label': 1},
 {'guid': 118, 'text_a': 'must have for a professionals and for thise who want to become one.', 'text_b': None, 'label': 1},
 {'guid': 130, 'text_a': 'passionate articles', 'text_b': None, 'label': 1},
 {'guid': 142, 'text_a': 'funtastic collection of information.', 'text_b': None, 'label': 1},
 {'guid': 148, 'text_a': 'generally i really like it, but i feel like i get too many notifications.', 'text_b': None, '

Let's use `classification_report()` function to check the precision, recall, f1-score, and the accuracy.

In [34]:
print(classification_report(test['sentiment'], model_outputs, 
                            labels = [0, 1, 2], 
                            target_names = ['NEGATIVE', 'NEUTRAL', 'POSITIVE']))

              precision    recall  f1-score   support

    NEGATIVE       0.89      0.94      0.91       515
     NEUTRAL       0.87      0.66      0.75       598
    POSITIVE       0.96      0.99      0.97      3429

    accuracy                           0.94      4542
   macro avg       0.91      0.86      0.88      4542
weighted avg       0.94      0.94      0.94      4542



Based on the output above, the accuracy is 94% and the accuracy is considered a good result. The result of the precision, recall, and f1-score are also good especially for label `positive` and `negative` which is above 89%. For the result of the precision for label `neutral` is good and have no big differences between other labels. Meanwhile, for the result of the recall and f1-score for label `neutral` are still good though it's not better than the other labels.